In [ ]:
from IPython.display import HTML, display

html = """
<style>
#cam { border:3px solid black; width:480px; }
button { padding:10px; margin:5px; font-size:16px; }
#status { font-size:18px; font-weight:bold; margin-top:10px; }
#faceStatus { font-size:18px; font-weight:bold; margin-top:5px; }
#gallery img { border:2px solid black; }
</style>

<h3>Viola-Jones + Save + Recognize + Status Deteksi</h3>

<canvas id="cam"></canvas><br>

<div id="status">⛔ Status: Kamera belum aktif</div>
<div id="faceStatus">❌ Status Wajah: Belum terdeteksi</div><br>

<button onclick="startCam()">Open Camera</button>
<button onclick="stopCam()">Stop Camera</button>
<button onclick="detectFaceToggle()">Deteksi Wajah</button>
<button onclick="saveCurrentFace()">💾 Simpan Wajah</button>
<button onclick="recognizeToggle()">🧠 Recognize</button>

<h4>Wajah Tersimpan</h4>
<div id="gallery" style="display:flex; gap:15px; flex-wrap:wrap;"></div>

<script async src="https://docs.opencv.org/4.x/opencv.js"></script>

<script>
let video, stream, canvas, ctx;
let running = false;
let detecting = false;
let recognizing = false;
let cvReady = false;
let classifier = null;

let lastFaceCanvas = null;
let faceDetected = false;

function onOpenCvReady(){
    cvReady = true;

    let cascadeUrl = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml";

    let request = new XMLHttpRequest();
    request.open("GET", cascadeUrl, true);
    request.responseType = "arraybuffer";

    request.onload = function(){
        let data = new Uint8Array(request.response);
        cv.FS_createDataFile("/", "face.xml", data, true, false, false);
        classifier = new cv.CascadeClassifier();
        classifier.load("face.xml");
    };
    request.send();
}

async function startCam(){
  if(!cvReady || !classifier){
    alert("OpenCV belum siap!");
    return;
  }

  video = document.createElement("video");
  stream = await navigator.mediaDevices.getUserMedia({video:true});
  video.srcObject = stream;
  await video.play();

  canvas = document.getElementById("cam");
  ctx = canvas.getContext("2d");
  canvas.width = video.videoWidth;
  canvas.height = video.videoHeight;

  running = true;
  document.getElementById("status").innerHTML = "✅ Status: Kamera Aktif";
  detectLoop();
}

function stopCam(){
  running = false;
  faceDetected = false;

  if(stream) stream.getTracks().forEach(t => t.stop());

  document.getElementById("status").innerHTML = "⛔ Status: Kamera Mati";
  document.getElementById("faceStatus").innerHTML = "❌ Status Wajah: Tidak terdeteksi";
}

function detectFaceToggle(){
  detecting = !detecting;
  alert(detecting ? "Deteksi wajah ON" : "Deteksi wajah OFF");
}

function recognizeToggle(){
  recognizing = !recognizing;
  alert(recognizing ? "Recognize ON" : "Recognize OFF");
}

function saveCurrentFace(){
  if(!faceDetected || !lastFaceCanvas){
    alert("❌ Wajah belum terdeteksi, tidak bisa disimpan!");
    return;
  }

  let name = prompt("Masukkan nama:");
  if(!name) return;

  let imgData = lastFaceCanvas
                .getContext("2d")
                .getImageData(0,0,50,50).data;

  let saved = JSON.parse(localStorage.getItem("faces") || "[]");

  saved.push({
    name: name,
    pixels: Array.from(imgData),
    image: lastFaceCanvas.toDataURL()
  });

  localStorage.setItem("faces", JSON.stringify(saved));
  loadFaces();

  alert("✅ Wajah berhasil disimpan!");
}

function loadFaces(){
  let gallery = document.getElementById("gallery");
  gallery.innerHTML = "";

  let saved = JSON.parse(localStorage.getItem("faces") || "[]");

  saved.forEach(face => {
    let div = document.createElement("div");
    div.innerHTML = `<img src="${face.image}" width="50"><br><small>${face.name}</small>`;
    gallery.appendChild(div);
  });
}

function compareFaces(p1, p2){
  let diff = 0;
  for(let i=0; i<p1.length; i+=4){
    diff += Math.abs(p1[i] - p2[i]);
  }
  return diff;
}

async function detectLoop(){
  if(!running) return;

  ctx.drawImage(video, 0, 0);

  faceDetected = false;
  document.getElementById("faceStatus").innerHTML = "❌ Status Wajah: Tidak terdeteksi";

  if(detecting && classifier){
    let src = new cv.Mat(canvas.height, canvas.width, cv.CV_8UC4);
    src.data.set(ctx.getImageData(0,0,canvas.width,canvas.height).data);

    let gray = new cv.Mat();
    cv.cvtColor(src, gray, cv.COLOR_RGBA2GRAY);

    let faces = new cv.RectVector();
    let msize = new cv.Size(50,50);
    classifier.detectMultiScale(gray, faces, 1.1, 3, 0, msize);

    ctx.lineWidth = 3;
    ctx.strokeStyle = "red";

    if(faces.size() > 0){
      faceDetected = true;
      document.getElementById("faceStatus").innerHTML =
        "✅ Status Wajah: Terdeteksi (" + faces.size() + ")";
    }

    for(let i=0; i<faces.size(); i++){
      let f = faces.get(i);
      ctx.strokeRect(f.x, f.y, f.width, f.height);

      lastFaceCanvas = document.createElement("canvas");
      lastFaceCanvas.width = 50;
      lastFaceCanvas.height = 50;
      lastFaceCanvas.getContext("2d").drawImage(
        canvas, f.x, f.y, f.width, f.height,
        0,0,50,50
      );

      // ===== RECOGNITION FINAL =====
      if(recognizing && faceDetected){

        let current = lastFaceCanvas
                      .getContext("2d")
                      .getImageData(0,0,50,50).data;

        let saved = JSON.parse(localStorage.getItem("faces") || "[]");

        if(saved.length === 0){
          ctx.fillStyle = "yellow";
          ctx.font = "18px Arial";
          ctx.fillText("Belum ada data", f.x, f.y-10);
        }
        else {
          let bestName = "Tidak Diketahui";
          let minDiff = Infinity;

          saved.forEach(face => {
            let d = compareFaces(face.pixels, current);
            if(d < minDiff){
              minDiff = d;
              bestName = face.name;
            }
          });

          if(minDiff < 150000){
            ctx.fillStyle = "lime";
            ctx.font = "18px Arial";
            ctx.fillText(bestName, f.x, f.y-10);
          } else {
            ctx.fillStyle = "yellow";
            ctx.font = "18px Arial";
            ctx.fillText("Tidak Diketahui", f.x, f.y-10);
          }
        }
      }
    }

    src.delete();
    gray.delete();
    faces.delete();
  }

  requestAnimationFrame(detectLoop);
}

window.onload = loadFaces;
</script>

<script>
cv['onRuntimeInitialized'] = onOpenCvReady;
</script>
"""

display(HTML(html))
